In [1]:
import numpy as np

In [ ]:
@njit
def v3_decode_prod_plan(ZPK, QPK, demand):
    """
    Decodes a production plan from swarm-algorithm position
    -----
    Considers four cases for each period t and product i:
      
      1. No production before or after t  --> meet current demand; incentivize swarm to avoid this case 
      2. Production after but not before t --> produce until next, preproduce ahead for t2 to t3
      3. Production before but not after t --> produce rest (1-preproduced ) share of demand from t to T
      4. Production both before and after t --> produce rest of demand from t to t2, preproduce ahead for t2 to t3
    
    Parameters
    ----------
    ZPK : 2D ndarray of int
        Binary production decision matrix, shape (M, T).
    QPK : 2D ndarray of float
        Preproduction fractions, shape (M, T).
    demand : 2D ndarray of float
        Demand per product per period, shape (M, T).

    Returns
    -------
    outT : 2D ndarray of float
        Production quantities per period per product, shape (T, M).
    """
    T = ZPK.shape[1]
    M = ZPK.shape[0]
    XPK = np.zeros((M, T))
    
    
    for i in range(M):
        for t in range(T):
            # skip non production periods
            if ZPK[i, t] == 0:
                continue
            else:
                # Look backwards for a prior production period t1
                t1 = 0
                for j in range(-1, -1, -1):
                    if ZPK[i, j] == 1:
                        t1 = j
                        break
                # Find next production period t2
                t2 = T - 1
                for j in range(t+1, T):
                    if ZPK[i, j] == 1:
                        t2 = j
                        break
                # ---------------------------------------------------------------------------------------------------------- #
                if (t1==t or ZPK[i,t1]==0) and (t2==t or ZPK[i,t2]==0):
                    XPK[i, t] = demand[i,t]

                elif (t1==t or ZPK[i,t1]==0):
                    t3 = T - 1
                    for j in range(t2+1, T):
                        if ZPK[i, j] == 1:
                            t3 = j
                            break
                            
                    end = t3 + 1 if (ZPK[i, t3] == 0 or t3 == t2) else t3
                    s1 = 0.0
                    for j in range(t2, end):
                        s1 += demand[i, j]
                        
                    s2 = 0.0
                    for j in range(t, t2):
                        s2 += demand[i, j]
    
                    XPK[i, t] = round(s2 + QPK[i, t2] * s1)

                elif (t2==t or ZPK[i,t2]==0):
                    s = 0.0
                    for j in range(t, t2+1):
                        s += demand[i, j]
                    
                    XPK[i, t] = round((1 - QPK[i, t]) * s)

                else:
                    t3 = T - 1
                    for j in range(t2+1, T):
                        if ZPK[i, j] == 1:
                            t3 = j
                            break
                    end = t3 + 1 if (ZPK[i, t3] == 0 or t3 == t2) else t3
    
                    s1 = 0.0
                    for j in range(t2, end):
                        s1 += demand[i, j]
    
                    s2 = 0.0
                    for j in range(t, t2):
                        s2 += demand[i, j]
    
                    XPK[i, t] = round((1-QPK[i, t]) * s2 + QPK[i, t2] * s1)


    # Build the output array and remove negative values, then transpose.
    out = np.empty((M, T))
    for i in range(M):
        for t in range(T):
            if XPK[i, t] < 0:
                out[i, t] = 0.0
            else:
                out[i, t] = XPK[i, t]
    outT = np.empty((T, M))
    for t in range(T):
        for i in range(M):
            outT[t, i] = out[i, t]
    return outT

       

In [ ]:
@njit
def v2_decode_prod_plan(ZPK, QPK, demand):
    """
    Decodes a production plan from swarm-algorithm position
    -----
    Considers eight cases for each period t and product i:
      1 t = first period
      1.1. No future production after period 1--> meet current demand only.
      1.2 Future production planned after period 1 --> meet until next, preproduce if needed for t2 to t3
      2 t = final period
      2.1 At last period, no prior production --> meet current demand.
      2.1. At last period with prior production --> meet rest of current demand
      3 t in [2,T-1]
      3.1 No production before or after t  --> meet current demand.
      3.2 Production before but not after t --> produce rest of demand from t to T
      3.3 Production after but not before t --> produce until next, preproduce ahead for t2 to t3
      3.4 Production both before and after t --> produce rest of demand from t to t2, preproduce ahead for t2 to t3
    
    Parameters
    ----------
    ZPK : 2D ndarray of int
        Binary production decision matrix, shape (M, T).
    QPK : 2D ndarray of float
        Preproduction fractions, shape (M, T).
    demand : 2D ndarray of float
        Demand per product per period, shape (M, T).

    Returns
    -------
    outT : 2D ndarray of float
        Production quantities per period per product, shape (T, M).
    """
    T = ZPK.shape[1]
    M = ZPK.shape[0]
    XPK = np.zeros((M, T))
    
    
    for i in range(M):
        for t in range(T):
            # skip non production periods
            if ZPK[i, t] == 0:
                continue
            else:
                
                # Find next production period t2
                t2 = T - 1
                for j in range(t+1, T):
                    if ZPK[i, j] == 1:
                        t2 = j
                        break
                # ---------------------------------------------------------------------------------------------------------- #
                # t = 1
                if t == 0:
                    # Case 1: If no production is planned in the future, meet just the current demand 
                    if (t2 == T - 1) and (ZPK[i, t2] == 0):
                        
                        XPK[i, t] = demand[i,t]
                        
                    # Case 2: If production is planned in the future, meet all the demand until the next production period and preproduce if necessary
                    else:
                        # find production period after the next production period
                        t3 = T - 1
                        for j in range(t2+1, T):
                            if ZPK[i, j] == 1:
                                t3 = j
                                break

                        end = t3 + 1 if (ZPK[i, t3] == 0 or t3 == t2) else t3
   
                        s1 = 0.0
                        for j in range(t2, end):
                            s1 += demand[i, j]
                            
                            
                        s2 = 0.0                       
                        for j in range(t, t2):
                            s2 += demand[i, j]
                            
                        XPK[i, t] = round(s2 + QPK[i, t2] * s1)
                        
                # ---------------------------------------------------------------------------------------------------------- #
                # t = T
                elif t == T - 1:
                    
                    # Look backwards for a prior production period t1
                    t1 = 0
                    for j in range(t-1, -1, -1):
                        if ZPK[i, j] == 1:
                            t1 = j
                            break
                            
                    # Case 3: no prior production before the last period --> meet all demand now
                    if (t1==0) and (ZPK[i,t1]==0):
                        s = 0.0
                        for j in range(t1, t+1):
                            s += demand[i, j]
                        XPK[i, t] = demand[i,t]
                        
                    # Case 4: prior production --> meet  backordered demand from t1 and current demand
                    else:
                        XPK[i, t] = round((1-QPK[i, t]) * demand[i, t])
                          
                # ----------------------------------------------------------------------------------------------------------#
                # t in [2,T-1]
                else:
                    # Look backwards for a prior production period t1
                    t1 = 0
                    for j in range(t-1, -1, -1):
                        if ZPK[i, j] == 1:
                            t1 = j
                            break

                    # Case 5: no production before AND after t
                    if (ZPK[i,t1]==0) and (ZPK[i,t2]==0):
                        XPK[i, t] = demand[i,t]
                        
                        
                    # Case 6: production before but not after t              
                    elif (ZPK[i,t1]==1) and (ZPK[i,t2]==0):
                        s = 0.0
                        for j in range(t, t2+1):
                            s += demand[i, j]
                        
                        XPK[i, t] = round((1 - QPK[i, t]) * s)

                    # Case 7: production after but not before t 
                    elif (ZPK[i,t1]==0) and (ZPK[i,t2]==1):
                        t3 = T - 1
                        for j in range(t2+1, T):
                            if ZPK[i, j] == 1:
                                t3 = j
                                break
                                
                        end = t3 + 1 if (ZPK[i, t3] == 0 or t3 == t2) else t3
                        s1 = 0.0
                        for j in range(t2, end):
                            s1 += demand[i, j]
                            
                        s2 = 0.0
                        for j in range(t, t2):
                            s2 += demand[i, j]

                        XPK[i, t] = round(s2 + QPK[i, t2] * s1)
                        
                        
                    # Case 8: production after and before t  
                    else:
                        t3 = T - 1
                        for j in range(t2+1, T):
                            if ZPK[i, j] == 1:
                                t3 = j
                                break
                        end = t3 + 1 if (ZPK[i, t3] == 0 or t3 == t2) else t3
   
                        s1 = 0.0
                        for j in range(t2, end):
                            s1 += demand[i, j]
   
                        s2 = 0.0
                        for j in range(t, t2):
                            s2 += demand[i, j]

                        XPK[i, t] = round((1-QPK[i, t]) * s2 + QPK[i, t2] * s1)
                    

    # Build the output array and remove negative values, then transpose.
    out = np.empty((M, T))
    for i in range(M):
        for t in range(T):
            if XPK[i, t] < 0:
                out[i, t] = 0.0
            else:
                out[i, t] = XPK[i, t]
    outT = np.empty((T, M))
    for t in range(T):
        for i in range(M):
            outT[t, i] = out[i, t]
    return outT


In [ ]:
@njit
def v1_decode_prod_plan(ZPK, QPK, demand):
    T = ZPK.shape[1]
    M = ZPK.shape[0]
    XPK = np.zeros((M, T))
    # iterate products
    for i in range(M):
        for t in range(T):
            # skip non production periods
            if ZPK[i, t] == 0:
                continue
            else:
                
                # Find next production period t2
                t2 = T - 1
                for j in range(t+1, T):
                    if ZPK[i, j] == 1:
                        t2 = j
                        break
                # ---------------------------------------------------------------------------------------------------------- #
                # t = 1
                if t == 0:
                    # Case 1: If no production is planned in the future, meet all remaining demand now
                    if (t2 == T - 1) and (ZPK[i, t2] == 0):
                        
                        s = 0.0
                        for j in range(t, T):
                            s += demand[i, j]
                        XPK[i, t] = demand[i,t]
                        
                    # Case 2: If production is planned in the future, meet all the demand until the next production period
                    else:
                        XPK[i, t] = (1 + min(QPK[i, t],0)) * demand[i, t] + max(QPK[i, t2] * demand[i, t2],0)
        
                        s = 0.0
                        
                        for j in range(t+1, t2):
                            s += demand[i, j]
                            
                        XPK[i, t] = round(XPK[i, t] + s)
                        
                # ---------------------------------------------------------------------------------------------------------- #
                # t = T
                elif t == T - 1:
                    
                    # Look backwards for a prior production period t1
                    t1 = 0
                    for j in range(t-1, -1, -1):
                        if ZPK[i, j] == 1:
                            t1 = j
                            break
                            
                    # Case 3: no prior production before the last period --> meet all demand now
                    if (t1==0) and (ZPK[i,t1]==0):
                        s = 0.0
                        for j in range(t1, t+1):
                            s += demand[i, j]
                        XPK[i, t] = demand[i,t]
                        
                        
                    # Case 4: prior production --> meet  backordered demand from t1 and current demand
                    else:
                        XPK[i, t] = round((1-QPK[i, t]) * demand[i, t] - min(QPK[i, t1] * demand[i, t1],0))
                        #XPK[i, t] = round((1-max(0,QPK[i, t])) * demand[i, t] - min(QPK[i, t1] * demand[i, t1],0))

                       
                # ----------------------------------------------------------------------------------------------------------#
                # t in [2,T-1]
                else:
                    # Look backwards for a prior production period t1
                    t1 = 0
                    for j in range(t-1, -1, -1):
                        if ZPK[i, j] == 1:
                            t1 = j
                            break

                    
                    # Case 5: no production before AND after t --> meet all demand now
                    if (ZPK[i,t1]==0) and (ZPK[i,t2]==0):
                        s = 0.0
                        for j in range(t1, t2+1):
                            s += demand[i, j]
                        XPK[i, t] = demand[i,t]
                        
                        
                    # Case 6: production before but not after t --> produce backorders and then produce everything for t to T               
                    elif (ZPK[i,t1]==1) and (ZPK[i,t2]==0):
                        
                        
                        XPK[i, t] = (1 - QPK[i, t]) * demand[i, t] - min(QPK[i, t1] * demand[i, t1],0)

                        s = 0.0
                        for j in range(t+1, t2+1):
                            s += demand[i, j]
                        
                        XPK[i, t] = round(XPK[i, t] + s)

                    # Case 7: production after but not before t -->  produce everything for t1 to t and preproduce t+1 to t2
                    elif (ZPK[i,t1]==0) and (ZPK[i,t2]==1):
                        
                        XPK[i, t] = (1 + QPK[i, t]) * demand[i, t] + max(QPK[i, t2] * demand[i, t2],0)
                        #XPK[i, t] = (1 + min(0,QPK[i, t])) * demand[i, t] + max(QPK[i, t2] * demand[i, t2],0)
                        
                        s1 = 0.0
                        for j in range(t1, t):
                            s1 += demand[i, j]
                        
                        s2 = 0.0
                        for j in range(t+1, t2):
                            s2 += demand[i, j]
                         
                        XPK[i, t] = round(XPK[i, t] + s1 + s2)
                        
                    # Case 8: production after and before t --> produce in advance t to t2 and cover backorders of t1 
                    else:
                        
                            
                        XPK[i, t] = (1-abs(QPK[i, t])) * demand[i, t] + max(QPK[i, t2] * demand[i, t2],0) - min(QPK[i, t1] *demand[i, t1],0)

                            
                        s = 0.0
                        for j in range(t+1, t2):
                            s += demand[i, j]

                        XPK[i, t] = round(XPK[i, t] + s)
                    

    # Build the output array and apply np.maximum with 0, then transpose.
    out = np.empty((M, T))
    for i in range(M):
        for t in range(T):
            if XPK[i, t] < 0:
                out[i, t] = 0.0
            else:
                out[i, t] = XPK[i, t]
    outT = np.empty((T, M))
    for t in range(T):
        for i in range(M):
            outT[t, i] = out[i, t]
    return outT


In [ ]:
# this version works perfectly fine  , go back to it if you messed up


@njit
def v0_decode_prod_plan(ZPK, QPK, demand):
    T = ZPK.shape[1]
    M = ZPK.shape[0]
    XPK = np.zeros((M, T))
    # iterate products
    for i in range(M):
        for t in range(T):
            # skip non production periods
            if ZPK[i, t] == 0:
                continue
            else:
                
                # Find next production period t2
                t2 = T - 1
                for j in range(t+1, T):
                    if ZPK[i, j] == 1:
                        t2 = j
                        break
                # ---------------------------------------------------------------------------------------------------------- #
                # t = 1
                if t == 0:
                    # Case 1: If no production is planned in the future, meet all remaining demand now
                    if (t2 == T - 1) and (ZPK[i, t2] == 0):
                        
                        s = 0.0
                        for j in range(t, T):
                            s += demand[i, j]
                        XPK[i, t] = s
                        
                    # Case 2: If production is planned in the future, meet all the demand until the next production period
                    else:
                        XPK[i, t] = (1 + min(QPK[i, t],0)) * demand[i, t] + max(QPK[i, t2] * demand[i, t2],0)
        
                        s = 0.0
                        
                        for j in range(t+1, t2):
                            s += demand[i, j]
                            
                        XPK[i, t] = round(XPK[i, t] + s)
                        
                # ---------------------------------------------------------------------------------------------------------- #
                # t = T
                elif t == T - 1:
                    
                    # Look backwards for a prior production period t1
                    t1 = 0
                    for j in range(t-1, -1, -1):
                        if ZPK[i, j] == 1:
                            t1 = j
                            break
                            
                    # Case 3: no prior production before the last period --> meet all demand now
                    if (t1==0) and (ZPK[i,t1]==0):
                        s = 0.0
                        for j in range(t1, t+1):
                            s += demand[i, j]
                        XPK[i, t] = s
                        
                        
                    # Case 4: prior production --> meet  backordered demand from t1 and current demand
                    else:
                        XPK[i, t] = round((1-QPK[i, t]) * demand[i, t] - min(QPK[i, t1] * demand[i, t1],0))
                        #XPK[i, t] = round((1-max(0,QPK[i, t])) * demand[i, t] - min(QPK[i, t1] * demand[i, t1],0))

                       
                # ----------------------------------------------------------------------------------------------------------#
                # t in [2,T-1]
                else:
                    # Look backwards for a prior production period t1
                    t1 = 0
                    for j in range(t-1, -1, -1):
                        if ZPK[i, j] == 1:
                            t1 = j
                            break

                    
                    # Case 5: no production before AND after t --> meet all demand now
                    if (ZPK[i,t1]==0) and (ZPK[i,t2]==0):
                        s = 0.0
                        for j in range(t1, t2+1):
                            s += demand[i, j]
                        XPK[i, t] = s
                        
                        
                    # Case 6: production before but not after t --> produce backorders and then produce everything for t to T               
                    elif (ZPK[i,t1]==1) and (ZPK[i,t2]==0):
                        
                        
                        XPK[i, t] = (1 - QPK[i, t]) * demand[i, t] - min(QPK[i, t1] * demand[i, t1],0)

                        s = 0.0
                        for j in range(t+1, t2+1):
                            s += demand[i, j]
                        
                        XPK[i, t] = round(XPK[i, t] + s)

                    # Case 7: production after but not before t -->  produce everything for t1 to t and preproduce t+1 to t2
                    elif (ZPK[i,t1]==0) and (ZPK[i,t2]==1):
                        
                        XPK[i, t] = (1 + QPK[i, t]) * demand[i, t] + max(QPK[i, t2] * demand[i, t2],0)
                        #XPK[i, t] = (1 + min(0,QPK[i, t])) * demand[i, t] + max(QPK[i, t2] * demand[i, t2],0)
                        
                        s1 = 0.0
                        for j in range(t1, t):
                            s1 += demand[i, j]
                        
                        s2 = 0.0
                        for j in range(t+1, t2):
                            s2 += demand[i, j]
                         
                        XPK[i, t] = round(XPK[i, t] + s1 + s2)
                        
                    # Case 8: production after and before t --> produce in advance t to t2 and cover backorders of t1 
                    else:
                        
                            
                        XPK[i, t] = (ZPK[i, t]-abs(QPK[i, t])) * demand[i, t] + max(QPK[i, t2] * demand[i, t2],0) - min(QPK[i, t1] *demand[i, t1],0)

                            
                        s = 0.0
                        for j in range(t+1, t2):
                            s += demand[i, j]

                        XPK[i, t] = round(XPK[i, t] + s)
                    

    # Build the output array and apply np.maximum with 0, then transpose.
    out = np.empty((M, T))
    for i in range(M):
        for t in range(T):
            if XPK[i, t] < 0:
                out[i, t] = 0.0
            else:
                out[i, t] = XPK[i, t]
    outT = np.empty((T, M))
    for t in range(T):
        for i in range(M):
            outT[t, i] = out[i, t]
    return outT

